# Stochastic semi-nonparametric envelopment of z variables data (StoNEZD)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 16, 2020

### Objective: Estimating a log-transformed cost function model with z-variable and technicial inefficiency

In [1]:
# import package pystoned
from pystoned import *

import pandas as pd
import numpy as np

### Import data

In [2]:
# import execl
df = pd.read_excel ('data.xlsx')

In [3]:
# output (total cost)
y  = df['TOTEX']
y  = y.to_numpy()
y  = y.tolist()

In [4]:
# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)
x   = x.tolist()

In [5]:
# Z variables
z = df['PerUndGr']
z = z.to_numpy()
z = z.tolist()

### Stage 1: Convex Nonparametric Least Squares estimation

In [6]:
# import the CNLSZ module
crt   = "mult"
func  = "cost"
pps   = "crs"

model = CNLSZ.cnlsz(y, x, z, crt, func, pps)

In [7]:
# slove the model
from pyomo.environ import *
from pyomo.opt import SolverFactory

solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [8]:
# display estimates
model.e.display() 
model.b.display() 
model.d.display()

e : residuals
    Size=89, Index=i
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
      0 :  None :    0.06655978564927588 :  None : False : False :  Reals
      1 :  None :   0.026027226814423536 :  None : False : False :  Reals
      2 :  None :    0.04364801103435727 :  None : False : False :  Reals
      3 :  None :  -0.008867997515358808 :  None : False : False :  Reals
      4 :  None :    0.18923052147582783 :  None : False : False :  Reals
      5 :  None :   -0.11484869617623955 :  None : False : False :  Reals
      6 :  None :  -0.015960371273761178 :  None : False : False :  Reals
      7 :  None :     0.1571223212709854 :  None : False : False :  Reals
      8 :  None :     0.2275942254642831 :  None : False : False :  Reals
      9 :  None :   -0.08712594899372214 :  None : False : False :  Reals
     10 :  None :     0.0344521600861926 :  None : False : False :  Reals
     11 :  None :   -0.07715597727604039 :  None : False : False :  Reals
   

In [9]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

### Stage 2: Residuals decomposition

In [10]:
# decomposition by method od moment (MoM)
method      = 'MoM'
TE          = StoNED.stoned(eps, func, method, crt)

In [11]:
# display technicial inefficiency
TE

array([1.03121493, 1.05821989, 1.04639386, 1.08203538, 0.95361498,
       1.15770204, 1.08694098, 1.00165531, 0.93055997, 1.13741235,
       1.05254899, 1.13020281, 0.93627354, 0.93209742, 1.11681622,
       1.14838999, 1.00193743, 0.91453346, 1.06479365, 1.143538  ,
       1.12467329, 1.28367287, 0.90737425, 1.15247634, 1.00181278,
       1.06247841, 0.92905192, 1.2206748 , 0.95626038, 1.09164409,
       1.11123761, 1.29828371, 1.09911   , 0.92965214, 1.09167274,
       1.00195347, 1.22598382, 1.20831039, 1.17581089, 1.0847533 ,
       1.07620148, 1.07165164, 1.00622662, 1.14936668, 1.15952411,
       1.17413103, 1.04900834, 1.00969686, 1.17437588, 1.19141198,
       0.92226046, 1.0531912 , 1.00307759, 0.9438099 , 1.02918898,
       1.23855826, 1.19505698, 1.10767467, 1.22257971, 1.09886745,
       1.183115  , 1.11828536, 1.10932594, 1.04443531, 0.89711913,
       1.13285104, 1.00370318, 1.12860974, 1.04627089, 1.28544212,
       1.09466016, 1.05638897, 1.17857304, 1.13774755, 1.16607